<a href="https://colab.research.google.com/github/Boin-Kau/recommender-systems-using-python/blob/main/Latent%20Factor%20Models/latent_factor_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Required Modules

<br>

- matplotlib >= 3.1.1
- mplcursors >= 0.5.1
    - use command to install(in notebook) : **!pip install mplcursors**
- numpy >= 1.21.5
- pandas >= 0.25.1
- sklearn >= 0.21.3

## Import Modules

In [1]:
!pip install mplcursors

     |████████████████████████████████| 88 kB 4.2 MB/s 
  Created wheel for mplcursors: filename=mplcursors-0.5.1-py3-none-any.whl size=20811 sha256=a867e2bec5030b79def488452e9cbb02ddb659ee56550cee4417852ccf7e5a7e
  Stored in directory: /root/.cache/pip/wheels/ae/cd/13/09fe660e3e9d392c7e9b46f202369cd9c2b8dddfb47d5950ef
Successfully built mplcursors


In [2]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors # Use this is for creating a cursor-interactive plot with "%matplotlib notebook"
from sklearn.decomposition import NMF # Use this for training Non-negative Matrix Factorization
from sklearn.utils.extmath import randomized_svd # Use this for training Singular Value Decomposition
from sklearn.manifold import TSNE # Use this for training t-sne manifolding
# my import
import os 

plt.style.use('ggplot') # You can also use different style

# just for plot checking, use this option
# %matplotlib inline

# for interactive plot
# If you use this option, plot will appear at first-drawn position
%matplotlib notebook

warnings.filterwarnings('ignore')

## Data Loading

<br>

> ### File description : MovieLens-100K
- movies.csv : list of movies (9742 movies)
- ratings.csv : list of ratings given by users (610 users, 100,836 ratings)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dir = 'drive/MyDrive/RecoSys/Data/MovieLens100K/'
# dir = './Data/MovieLens100K/'
df_ratings = pd.read_csv(dir + 'ratings.csv', usecols=['userId', 'movieId', 'rating'])
df_movies = pd.read_csv(dir + 'movies.csv', usecols=['movieId', 'title', 'genres']) # for title-matching

## Simple EDA (Exploratory Data Analysis)

<br>

- Before starting, let's perform simple data analysis on the given dataset

In [5]:
# 고유 사용자, 고유 영화 갯수 확인
n_users = len(df_ratings['userId'].unique())
n_movies = len(df_ratings['movieId'].unique())

n_users, n_movies # 610 명의 사용자가 9724개의 영화에 평점을 매김을 확인

(610, 9724)

In [6]:
len(df_movies) # len(df_movies) - n_movies 만큼의 평점이 매겨지지 않은 영화가 존재

9742

In [7]:
# ratings의 기술통계량 확인
df_ratings['rating'].describe()

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [19]:
# df_movies 데이터 탐색
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Generate Utility Matrix $A$

<br>

> ### Problem 1 (2 points)
1. Generate an utility matrix $A$ by using **df_ratings** (store rating values with unique 'movieId' and 'userId')
2. Within a matrix $A$, replace NaN values (unknown ratings) with 0 (zero value)
3. Convert the utility matrix $A$ to **numpy array**

In [9]:
"""
# Utility Matrix의 형태는 (n_movies, n_users)
# 즉 Utility Matrix 의 각 행은 movie, 각 열은 user를 나타냄
"""
# your code here
## Utility Matrix 만들기
A = df_ratings.pivot(index='movieId',
                     columns='userId',
                     values='rating')
## NaN 값을 0으로 대체하기
A = A.fillna(0)

## Utility Matrix를 Numpy array로 변환하기
A = A.to_numpy()

# 올바른 형태로 utility matrix가 생성되었는지 확인
print(A.shape)

(9724, 610)


## Training SVD Model

<br>

> ### Problem 2 (2 points)
1. Decompose utility matrix $A$ into three matrices $U$, $\sum$, and $V^T$ by training SVD model (you can use  randomized_svd() function provided from scikit-learn)
    - Refer to: https://scikit-learn.org/stable/modules/generated/sklearn.utils.extmath.randomized_svd.html

2. After training SVD is completed, perform dot-product of $U$, $\sum$, and $V^T$ to obtain the matrix $A_{approx_{svd}}$ that approximates matrix $A$
    - Note that, **$\sum$ should be a diagonal matrix**, not a vector
    - Before computing $A_{approx_{svd}}$, you will need to transform $\sum$ generated from randomized_svd() to a diagonal matrix

In [10]:
"""
# sklearn에서 제공하는 randomized_svd()를 통해 SVD 모델을 학습하여 U, Sigma, VT를 구함
# k는 분해될 행렬들의 feature/factor 의 크기를 정하는 hyperparameter
# 여러 인자를 조정해서 randomized_svd()를 실행 가능 (자세한 내용은 위의 참고 사이트를 참고)
# 분해된 행렬 U, Sigma, VT의 형태는 (n_movies, k), (k,) (k, n_users)
# 분해된 행렬들을 이용하여 dot-product 연산을 수행하면 원래의 utility matrix와 같은 (n_movies, n_users) 형태의 근사 행렬을 얻을 수 있음
"""

# your code here

k = 6
U, Sigma, VT = randomized_svd(A, 
                              n_components=k)

# 분해된 행렬이 올바른 형태로 생성되었는지 확인
print(U.shape, Sigma.shape, VT.shape)

# Transform to a diagonal matrix
Sigma = np.diag(Sigma)

A_approx_svd = np.matmul(np.matmul(U,Sigma),VT)

# 근사 행렬이 올바른 형태로 생성되었는지 확인
print(A_approx_svd.shape)

(9724, 6) (6,) (6, 610)
(9724, 610)


## Training NMF Model

<br>

> ### Problem 3 (2 points)
1. Decompose utility matrix $A$ into two matrices $W$ and $H$ by training NMF model (using NMF()).
    - Refer to: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html#sklearn.decomposition.NMF

2. After training NMF, perform dot-product of $W$ and $H$ to obtain the matrix $A_{approx_{nmf}}$ that approximates matrix $A$

In [11]:
"""
# sklearn에서 제공하는 NMF()를 통해 NMF 모델을 학습하여 W, H를 구함
# SVD와 마찬가지로, k는 분해될 행렬들의 feature/factor 의 크기를 정하는 hyperparameter
# 여러 인자를 조정해서 NMF() 모델을 생성 (자세한 내용은 위의 참고 사이트를 참고)
# 분해된 행렬 W와 H는 .fit(data)를 실행한 후에 구할수 있으며, W와 H의 형태는 (n_movies, k), (k, n_users)
# 분해된 행렬들을 이용하여 dot-product 연산을 수행하면 원래의 utility matrix와 같은 (n_movies, n_users) 형태의 근사 행렬을 구할 수 있음
"""

# your code here

k = 20
model_nmf = NMF(n_components=k)
model_nmf.fit(A)
W = model_nmf.transform(A)
H = model_nmf.components_


# 분해된 행렬이 올바른 형태로 생성되었는지 확인
print(W.shape, H.shape)

A_approx_nmf = np.matmul(W,H)

# 근사 행렬이 올바른 형태로 생성되었는지 확인
print(A_approx_nmf.shape)

(9724, 20) (20, 610)
(9724, 610)


## Compute loss by implementing a custom function

> ### Problem 4 (2 points)
1. Implement **compute_error(actual, prediction)** function that takes matrices 'actual' and 'prediction' as input parameters.
    1. Ignore zero values in the actual maxtrix
    2. Compute SSE and RMSE
    3. Return SSE and RMSE

<br>

2. Implement **compute_error_all(actual, prediction)** function that takes matrices 'actual' and 'prediction' as input parameters.
    1. Do not ignore zero values in the actual matrix (compute all values)
    2. Compute SSE and RMSE
    3. Return SSE and RMSE

In [12]:
"""
# 실제 utility matrix A와 SVD 를 통해 생성된 행렬 A_approx_svd 간의 오차를 계산
# 실제 utility matrix A와 NMF 를 통해 생성된 행렬 A_approx_nmf 간의 오차를 계산
# 오차 값을 계산하기 위해 함수 compute_error(actual, prediction)와 compute_error_all(actual, prediction)를 구현
# 강의시간에 배운 수식을 통해 함수 구현
"""

# your code here

def compute_error(actual, prediction):
    # 매개변수로 입력받은 actual 행렬 안의 0값을 갖는 원소들은 오차 계산에서 제외합니다.
    prediction = prediction[np.nonzero(actual)]
    actual = actual[np.nonzero(actual)]
    
    sse = np.sum((actual - prediction)**2)
    rmse = np.sqrt(np.mean(actual - prediction)**2)
    
    return sse, rmse

def compute_error_all(actual, prediction):
    # actual 행렬 안의 0값을 갖는 원소들도 포함해서 오차를 계산합니다.
    sse = np.sum((actual - prediction)**2)
    rmse = np.sqrt(np.mean(actual - prediction)**2)
    
    return sse, rmse

In [13]:
print(f"SVD Error(ignoring zero values): SSE = {compute_error(A, A_approx_svd)[0]}, RMSE = {compute_error(A, A_approx_svd)[1]}")
print(f"NMF Error(ignoring zero values): SSE = {compute_error(A, A_approx_nmf)[0]}, RMSE = {compute_error(A, A_approx_nmf)[1]}")

print('\n')

print(f"SVD Error(including all zero values): SSE = {compute_error_all(A, A_approx_svd)[0]}, RMSE = {compute_error_all(A, A_approx_svd)[1]}")
print(f"NMF Error(including all zero values): SSE = {compute_error_all(A, A_approx_nmf)[0]}, RMSE = {compute_error_all(A, A_approx_nmf)[1]}")

SVD Error(ignoring zero values): SSE = 719626.0258028958, RMSE = 2.30135800654843
NMF Error(ignoring zero values): SSE = 611714.2958105388, RMSE = 1.9534323361176116


SVD Error(including all zero values): SSE = 895682.6465493335, RMSE = 0.0009267767880847323
NMF Error(including all zero values): SSE = 765456.100241853, RMSE = 0.012457023844305989


## Predict missing (unknown) values in utility matrix $A$ for a specific user


> ### Problem 5 (2 points)

- 실제 평점과 예측 평점을 확인할 수 있는 dataframe 생성을 위해 makePredictions(actual, pred, user) 함수를 정의
- makePredictions() 함수는 user(사용자 번호, user index)를 통해 actual, pred에서 rated movies(seen movies), non-rated movies(unseen movies)를 추출
- 그 다음 앞서 정의했던 df_movies와 추출한 2개의 dataframe을 concat 해줄 것
- column mismatching이 일어날 수 있는데, 우선 dataframe을 모든 column과 concat한 후 불필요한 'movieId' column을 drop 해줄 것
- 이어서 실제로 본 영화 목록을 rated_movies로 정의하고, rating을 기준으로 내림차순 정렬 수행
- 마찬가지로 평점이 부여되지 않은 영화들(평점을 예측하고자하는 영화들)의 목록을 unrated_movies로 정의하고, dataframe의 index를 기준으로 오름차순 정렬 수행

- dataframe의 앞, 뒤를 보는 함수는 .head(), .tail()를 사용할 수 있지만, 중간을 볼 수 있는 함수는 존재하지 않음
- 따라서, 함수 findMiddle(dataframe)을 정의하고, 이 함수는 indexing을 통해 dataframe의 중간 위치를 보여줌
- findMiddle()의 return은 dataframe의 중간 10개 부분

In [86]:
# test code
test_A = A.copy()
test_svd = A_approx_svd.copy()
test_df_movies = df_movies.copy()

df_test_A = pd.DataFrame(test_A).iloc[:, 10] 
df_test_svd = pd.DataFrame(test_svd).iloc[:, 10] 

test_df_movies.set_index(keys=[test_df_movies.movieId])
df_test_svd = pd.DataFrame(df_test_svd).set_index(keys=[df_test_A.index])

recommender_df = pd.concat([df_test_A, df_test_svd, test_df_movies], axis=1)
recommender_df = recommender_df.drop('movieId', axis=1)
recommender_df.columns = ['actual','pred','title','genres']

rated_movies = recommender_df.copy()
unrated_movies = recommender_df[recommender_df.actual == 0]
rated_movies = rated_movies.sort_values(by='actual', ascending=False)
unrated_movies = unrated_movies.sort_index(ascending=True)
len(unrated_movies)

<class 'pandas.core.series.Series'>


In [87]:
# your code here

def makePredictions(actual, pred, user):
    # rated_movies, unrated_movies 생성을 하기 위한 사전작업 dataframe을 정의할 수 있습니다. (약 4~5줄)
    rated_movies_df_user = pd.DataFrame(actual).iloc[:, user] 
    prediction_df_user = pd.DataFrame(pred).iloc[:, user]
    copy_df_movies = df_movies.copy()

    copy_df_movies.set_index(keys=[copy_df_movies.movieId])
    prediction_df_user = pd.DataFrame(prediction_df_user).set_index(keys=[rated_movies_df_user.index])

    recommender_df = pd.concat([rated_movies_df_user, prediction_df_user, copy_df_movies], axis=1)
    recommender_df = recommender_df.drop('movieId', axis=1)
    recommender_df.columns = ['actual','pred','title','genres']
    
    # 실제로 본 영화 목록 (평점이 부여된 영화 목록)
    rated_movies = recommender_df.copy()
    rated_movies = rated_movies.sort_values(by='actual', ascending=False)
    
    
    # 예측할 영화 목록 (평점이 부여되지 않은 영화 목록)
    unrated_movies = recommender_df[recommender_df.actual == 0]
    unrated_movies = unrated_movies.sort_index(ascending=True)
    
    return rated_movies, unrated_movies


def findMiddle(dataframe):
    # dataframe의 중간 부분을 반환하는 함수입니다.
    # return은 dataframe의 중간 10개 부분들 입니다. (dataframe 형식)
    length = len(dataframe)
    start = round(length/2-5)
    middle_df = pd.DataFrame(dataframe).iloc[start:start+10]
    print(middle_df)

In [98]:

# 임의의 사용자를 1명 선정하고, 앞서 작성한 함수에 svd, nmf의 근사 행렬을 인자로 전달

# your code here

# 임의의 사용자 (정수)를 선택
userNumber = 10

prediction_with_rated_svd, prediction_with_unrated_svd = makePredictions(A, A_approx_svd, userNumber) # 실제 Utility Matrix와 svd를 통해 근사한 행렬 간의 비교
prediction_with_rated_nmf, prediction_with_unrated_nmf = makePredictions(A, A_approx_nmf, userNumber) # 실제 Utility Matrix와 nmf를 통해 근사한 행렬 간의 비교
print(len(prediction_with_rated_svd))
print(len(prediction_with_unrated_svd))

9742
9660


In [89]:
prediction_with_rated_svd.head(10)

,actual,pred,title,genres
510,5.0,2.213728,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
97,5.0,2.597822,Braveheart (1995),Action|Drama|War
398,5.0,2.427154,"Fugitive, The (1993)",Thriller
1321,5.0,0.302260,Palmetto (1998),Crime|Drama|Mystery|Romance|Thriller
1186,5.0,0.309511,"Simple Wish, A (1997)",Children|Fantasy
1290,5.0,0.329641,"Sweet Hereafter, The (1997)",Drama
1275,5.0,0.004782,Alien: Resurrection (1997),Action|Horror|Sci-Fi
838,5.0,0.416683,Days of Thunder (1990),Action|Drama|Romance
1083,5.0,0.205827,Scream (1996),Comedy|Horror|Mystery|Thriller
307,5.0,1.713317,Clear and Present Danger (1994),Action|Crime|Drama|Thriller


In [90]:
prediction_with_rated_nmf.head(10)

,actual,pred,title,genres
510,5.0,1.815877,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
97,5.0,2.429128,Braveheart (1995),Action|Drama|War
398,5.0,2.340607,"Fugitive, The (1993)",Thriller
1321,5.0,0.355979,Palmetto (1998),Crime|Drama|Mystery|Romance|Thriller
1186,5.0,0.519616,"Simple Wish, A (1997)",Children|Fantasy
1290,5.0,0.370168,"Sweet Hereafter, The (1997)",Drama
1275,5.0,0.025543,Alien: Resurrection (1997),Action|Horror|Sci-Fi
838,5.0,0.491352,Days of Thunder (1990),Action|Drama|Romance
1083,5.0,0.236094,Scream (1996),Comedy|Horror|Mystery|Thriller
307,5.0,1.664602,Clear and Present Danger (1994),Action|Crime|Drama|Thriller


In [91]:
prediction_with_unrated_svd.head(10)

,actual,pred,title,genres
0,0.0,1.433101,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,0.0,0.944353,Jumanji (1995),Adventure|Children|Fantasy
2,0.0,0.503017,Grumpier Old Men (1995),Comedy|Romance
3,0.0,0.089736,Waiting to Exhale (1995),Comedy|Drama|Romance
4,0.0,0.435190,Father of the Bride Part II (1995),Comedy
6,0.0,0.491170,Sabrina (1995),Comedy|Romance
7,0.0,0.100173,Tom and Huck (1995),Adventure|Children
8,0.0,0.143532,Sudden Death (1995),Action
10,0.0,0.903412,"American President, The (1995)",Comedy|Drama|Romance
11,0.0,0.080944,Dracula: Dead and Loving It (1995),Comedy|Horror


In [92]:
prediction_with_unrated_nmf.head(10)

,actual,pred,title,genres
0,0.0,1.335411,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,0.0,0.948064,Jumanji (1995),Adventure|Children|Fantasy
2,0.0,0.520373,Grumpier Old Men (1995),Comedy|Romance
3,0.0,0.091872,Waiting to Exhale (1995),Comedy|Drama|Romance
4,0.0,0.446696,Father of the Bride Part II (1995),Comedy
6,0.0,0.534880,Sabrina (1995),Comedy|Romance
7,0.0,0.108913,Tom and Huck (1995),Adventure|Children
8,0.0,0.127432,Sudden Death (1995),Action
10,0.0,0.891332,"American President, The (1995)",Comedy|Drama|Romance
11,0.0,0.052945,Dracula: Dead and Loving It (1995),Comedy|Horror


In [93]:
prediction_with_unrated_svd.tail(10)

,actual,pred,title,genres
9714,0.0,-0.000844,Dogman (2018),Crime|Drama
9715,0.0,-0.000724,Mamma Mia: Here We Go Again! (2018),Comedy|Romance
9716,0.0,-0.000965,Tag (2018),Comedy
9717,0.0,-0.000965,The Man Who Killed Don Quixote (2018),Adventure|Comedy|Fantasy
9718,0.0,-0.000844,Boundaries (2018),Comedy|Drama
9719,0.0,-0.000965,Spiral (2018),Documentary
9720,0.0,-0.000844,Mission: Impossible - Fallout (2018),Action|Adventure|Thriller
9721,0.0,-0.000844,SuperFly (2018),Action|Crime|Thriller
9722,0.0,-0.000844,Iron Soldier (2010),Action|Sci-Fi
9723,0.0,-0.003839,BlacKkKlansman (2018),Comedy|Crime|Drama


In [94]:
prediction_with_unrated_nmf.tail(10)

,actual,pred,title,genres
9714,0.0,0.000000,Dogman (2018),Crime|Drama
9715,0.0,0.000000,Mamma Mia: Here We Go Again! (2018),Comedy|Romance
9716,0.0,0.000000,Tag (2018),Comedy
9717,0.0,0.000000,The Man Who Killed Don Quixote (2018),Adventure|Comedy|Fantasy
9718,0.0,0.000000,Boundaries (2018),Comedy|Drama
9719,0.0,0.000000,Spiral (2018),Documentary
9720,0.0,0.000000,Mission: Impossible - Fallout (2018),Action|Adventure|Thriller
9721,0.0,0.000000,SuperFly (2018),Action|Crime|Thriller
9722,0.0,0.000000,Iron Soldier (2010),Action|Sci-Fi
9723,0.0,0.000621,BlacKkKlansman (2018),Comedy|Crime|Drama


In [95]:
findMiddle(prediction_with_unrated_svd)

      actual      pred                                      title  \
4889     0.0 -0.034892                    Good bye, Lenin! (2003)   
4890     0.0 -0.021715                             Hidalgo (2004)   
4891     0.0  0.021471                     Starsky & Hutch (2004)   
4892     0.0 -0.060049                      Reckoning, The (2004)   
4893     0.0 -0.074653                             Persona (1966)   
4894     0.0 -0.056179  Passion of Anna, The (Passion, En) (1969)   
4895     0.0  0.004195            Corbeau, Le (Raven, The) (1943)   
4896     0.0 -0.010365              Pickup on South Street (1953)   
4897     0.0 -0.012043   Damned, The (La Caduta degli dei) (1969)   
4898     0.0 -0.035931                Just One of the Guys (1985)   

                            genres  
4889                  Comedy|Drama  
4890               Adventure|Drama  
4891  Action|Comedy|Crime|Thriller  
4892                   Crime|Drama  
4893                         Drama  
4894              

In [96]:
findMiddle(prediction_with_unrated_nmf)

      actual      pred                                      title  \
4889     0.0  0.001940                    Good bye, Lenin! (2003)   
4890     0.0  0.000000                             Hidalgo (2004)   
4891     0.0  0.020611                     Starsky & Hutch (2004)   
4892     0.0  0.031819                      Reckoning, The (2004)   
4893     0.0  0.039956                             Persona (1966)   
4894     0.0  0.000000  Passion of Anna, The (Passion, En) (1969)   
4895     0.0  0.005551            Corbeau, Le (Raven, The) (1943)   
4896     0.0  0.000000              Pickup on South Street (1953)   
4897     0.0  0.000000   Damned, The (La Caduta degli dei) (1969)   
4898     0.0  0.000000                Just One of the Guys (1985)   

                            genres  
4889                  Comedy|Drama  
4890               Adventure|Drama  
4891  Action|Comedy|Crime|Thriller  
4892                   Crime|Drama  
4893                         Drama  
4894              

## Visualize movie embeddings using T-SNE


> ### Extra Credit (2 points)

- NMF에서 분해되어 나온 행렬 W는 movie에 대한 잠재 표현(latent representation)을 갖고 있음
    - 마찬가지로, SVD에서 분해되어 나온 행렬 U도 movie에 대한 잠재 표현을 가짐
- 잠재 공간에서 실제 영화들이 NMF를 통해 어떻게 임베딩 되었는지 2차원 상으로 변환해 확인할 수 있으며, 대표적인 방법으로 t-sne을 사용할 수 있음
- 여러 인자를 조정해서 TSNE() 모델을 생성 가능 (자세한 내용은 공식 아래 사이트 참고)
    - t-sne 참고 사이트: https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html
    - t-sne 학습 과정에 시간이 다소 소요될 수 있음
- t-sne에서 fit()된 결과를 W_embedded에 저장
- W_embedded.embedding_ 으로 변환된 임베딩 결과물을 확인

In [ ]:
# your code here

model_tsne = 2
W_embedded = TSNE(n_components=model_tsne).fit_transform()

# 실제 2차원으로 변환되었는지 확인
# 변환된 결과는 (n_movies, 2)의 형태
print(W_embedded.embedding_.shape)

AttributeError: ignored

- Plot하기 위해 각 영화 임베딩과 movie dataframe(df_movies)을 결합
- 하지만 위 EDA에서 알 수 있듯이, movie.csv(df_movies)엔 사용자들이 평점을 매기지 않은 영화들이 존재함
- 따라서, 임베딩과 각 영화의 index를 맞추기 위한 작업이 필요함

- 먼저, numpy array로 변환 안된 (즉 dataframe 형태인) Utility matrix를 load하고, index들을 list로 추출
- df_movies에서 'movieId'가 추출한 list에 있는 경우만을 추출 (그러면 이제 df_movies엔 평점이 매겨진 영화들만이 남게 됨)

- (주의! 이때 reset_index(drop=True) 옵션을 사용해줘야 임베딩과 df_movies를 올바르게 결합할 수 있음)
- (그렇지 않으면 index-mismatching이 발생해 NaN값이 생성됨)

- 추출된 내용을 movie_with_embedding에 대입
- movie_with_embedding의 각 끝 열에 앞서 구한 임베딩을 열로 추가
- embedding은 (n_movies, 2)의 형태이므로, 각 column을 1개씩 'tsne1', 'tsne2'로써 movie_with_embedding의 column으로 추가
- 삽입 시, 형태를 맞추기 위해선 .reshape() method가 필요함

In [ ]:
# your code here

# 앞서 정의헀던 Utility Matrix와 동일한 방법으로 A2를 정의 (numpy array로 변환할 필요 없음)
A2 = df_ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)

# allList엔 A2의 index 값(즉, movieId)들을 list로 변환한 내용이 들어있음
allList =

# movie_with_embedding은 df_movies에서 allList와 일치하는 부분만을 가지게 됨
# movie_with_embedding에 'tsne1', 'tsne2' column을 추가 (추가하는 내용은 각각 임베딩의 첫번째 column, 두번째 column)
movie_with_embedding = 
movie_with_embedding['tsne1'] = 
movie_with_embedding['tsne2'] = 

In [ ]:
# 최종적으로 Plotting을 위해 생성한 dataframe이 어떤 모습으로 생겼는지 확인
# 이때, dataframe에 NaN값이 있어서는 안됨
movie_with_embedding

- tsne1, tsne2를 통해 scatter plot을 그릴 것
- 'mplcursors'를 통해 각 point마다 확인하고 싶은 정보를 labelling 해줄 것
- 이 기능을 사용하게 되면 point마다 text를 plot해줄 필요가 없고, 마우스 커서 클릭으로 point의 정보를 볼 수 있음
- 상단 예시 코드에서 YOUR_ANNOTATION_LIST를 적절히 선택해 각 point마다의 제목 또는 장르를 확인할 수 있음
- 이 함수를 사용하기 위해서는 '%matplotlib notebook' 이 필요함

```python
mplcursors.cursor(multiple = True).connect(
    "add", lambda sel: sel.annotation.set_text(
          YOUR_ANNOTATION_LIST[sel.target.index]
))
```

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10] # you can change size for your style
plt.xlim(movie_with_embedding['tsne1'].min(), movie_with_embedding['tsne1'].max()) # 축 범위 조정
plt.ylim(movie_with_embedding['tsne2'].min(), movie_with_embedding['tsne2'].max()) # 축 범위 조정

# your code here

# Scatter plot을 그리기


# 위의 mplcursors 코드를 이용해 각 point마다 labelling을 할 수 있습니다.
# YOUR_ANNOTATION_LIST는 영화 제목이 되어도 좋고, 영화 장르가 되어도 좋습니다.
# (직관성을 위해 '영화장르'로 labelling 하는 것을 추천합니다.)

plt.title('t-sne result (visualization of movie embeddings)')
plt.show()